In [1]:
import pandas as pd 
import os
from shutil import copyfile

In [2]:
solutions = pd.read_csv('training_solutions_rev1.zip')

In [3]:
IDs = solutions['GalaxyID']
dataLength = IDs.shape[0]
dataLength

61578

In [4]:
SOURCE = 'images_training_rev1/'
ELIPTICAL = 'eliptical/'
SPIRAL = 'spiral/'
IRREGULAR = 'irregular/'

VALIDATION = "images_validation/"
TRAINING = "images_training/"


In [5]:
ratio = 0.85

trainLenght = int(ratio*dataLength)

In [22]:
def copyImages(index,SOURCE,TARGET,TYPE):
    filename = str(solutions['GalaxyID'][index]) + ".jpg"
    file = SOURCE + filename
    distination = TARGET + TYPE + filename
    copyfile(file,distination)

def filterImages(index,SOURCE,TARGET):
    if solutions['Class6.1'][index]>0.61:
        copyImages(index,SOURCE,TARGET,IRREGULAR)
        
    elif solutions['Class4.1'][index] > 0.53:
        copyImages(index,SOURCE,TARGET,SPIRAL)
    
    else:
        copyImages(index,SOURCE,TARGET,ELIPTICAL)





for index in range(trainLenght):
    filterImages(index,SOURCE,TRAINING)

for index in range(trainLenght,dataLength):
    filterImages(index,SOURCE,VALIDATION)
    
    

In [6]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [62]:
import numpy as np
from PIL import Image
im = Image.open("images_training\irregular\\101133.jpg", "r")
size = im.size
print(size)
data = list(im.getdata())
data = [list(x) for x in data]
data = np.array(data)
data = data.reshape(424,424,3)
print(data.shape)


(424, 424)
(424, 424, 3)


In [105]:
def getData(index):
    img = Image.open("images_training_rev1/"+str(solutions['GalaxyID'][index])+ ".jpg", "r")
    data = list(img.getdata())
    data = [list(x) for x in data]
    data = np.array(data)
    data = data.reshape((1,424,424,3))
    return data

In [116]:
trainingData = np.empty((1,424,424,3),int)
testingData = np.empty((1,424,424,3),int)

for index in range(trainLenght):
    data = getData(index)
    print("Converting train : ",index)
    trainingData = np.append(trainingData,data,axis=0)

for index in range(trainLenght,dataLength):
    data = getData(index)
    print("Converting test : ",index)
    testingData = np.append(testingData,data,axis=0)

trainingData = trainingData[1:]
testingData = testingData[1:]


Converting train :  0
Converting train :  1
Converting train :  2
Converting train :  3
Converting train :  4
Converting train :  5
Converting train :  6
Converting train :  7
Converting train :  8
Converting train :  9
Converting train :  10
Converting train :  11
Converting train :  12
Converting train :  13
Converting train :  14
Converting train :  15
Converting train :  16
Converting train :  17
Converting train :  18
Converting train :  19
Converting train :  20
Converting train :  21
Converting train :  22
Converting train :  23
Converting train :  24
Converting train :  25
Converting train :  26
Converting train :  27
Converting train :  28
Converting train :  29
Converting train :  30
Converting train :  31
Converting train :  32
Converting train :  33
Converting train :  34
Converting train :  35
Converting train :  36
Converting train :  37
Converting train :  38
Converting train :  39
Converting train :  40
Converting train :  41
Converting train :  42
Converting train :  4

In [103]:
test = np.array([[2,2,2],[3,3,4],[3,4,5]])
print(test)
new = np.array([[4,5,5]])
newTest = np.append(test,new,axis=0)
print(newTest)

[[2 2 2]
 [3 3 4]
 [3 4 5]]
[[2 2 2]
 [3 3 4]
 [3 4 5]
 [4 5 5]]


In [41]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer=RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['acc'])

In [42]:
history = model.fit(train_generator, epochs=15, steps_per_epoch=90,validation_data=validation_generator, validation_steps=6)

Epoch 1/15


90/90 [==============================] - 52s 570ms/step - loss: 9.3378 - acc: 0.0989 - val_loss: 9.4444 - val_acc: 0.0927
Epoch 2/15
90/90 [==============================] - 69s 770ms/step - loss: 9.4288 - acc: 0.0920 - val_loss: 9.3935 - val_acc: 0.0880
Epoch 3/15
13/90 [===>..........................] - ETA: 34s - loss: 9.6000 - acc: 0.0892

KeyboardInterrupt: 